# Altair: Your Paper's Best Friend
There are a number of modules and packages that help you create charts and graphs and other visualizations in Python. Some popular ones include:
  * Matplotlib
  * Seaborn (sits on top of Matplotlib)
  * Bokeh
  * Altair
  
I have used a number of different approaches, and I've found Altair to be the most intuitive, quick to use, and aesthetically pleasing. So that's what we'll introduce today.

In [8]:
import pandas as pd
import altair as alt

## Visualizing VIA Reliability on the Kitchener Corridor
Using the outputs we created in the Pandas tutorial, we are going to produce some visualizations and summaries that help us communicate reliability on the Kitchener Corridor portion of the VIA Rail network. Let's load our data now






In [9]:
otp = pd.read_csv('kitchener_otp.csv')
times = pd.read_csv('kitchener_times.csv')

## Chart Objects
First, let's start with an introduction to Altair's `Chart` object. The `Chart` class holds the basis for creating a chart, and takes a Pandas dataframe as a data source. From there, we can mark 

In [10]:
alt.Chart(otp).mark_bar().encode(
    x='otp:Q',
    y='station:N',
    row='train:N'
)

alt.Chart(...)